This file is part of PIConGPU. \
Copyright 2024 Fabia Dietrich

# Preparing Insight data for PIConGPU

## Intro
This notebook allows you to manipulate Insight data and prepare it to be read via the InsightPulse profile into PIconGPU.
The raw Insight data cannot be used, since it (at least) has to be phase corrected and transformed into the time domain. 
Furthermore, the field can be propagated.

## Load modules

In [ ]:
# standard modules
import numpy as np
import matplotlib.pyplot as plt

# modules from preparingInsightData.py
from preparingInsightData import PrepRoutines

## Get the data
The far field data from an Insight measurement is stored in a h5 file and typically measured in dependence of two transversal coordinates ("x" and "y" in mm) and the frequency ("w" in rad/fs). If there are any deviations from this scheme, you will have to adjust the _PrepRoutines_ source code. 

Then, the first steps are:
1. read the far field data and store it in numpy arrays
2. fit the far field data intensity with a 2D gaussian to extract the beam center and waist size
3. propagate to the near field
4. fit the near field data with a 2D supergaussian to extract the beam center and waist size

For that, you need to provide path, filename and focal distance (same unit as the transversal scales) to the init function.

In [ ]:
foc = 2000  # mm, focal distance
insight = PrepRoutines("/put/your/path/here/", "filename.h5", foc)

In [ ]:
# some nice colorful pictures of your data
fig = plt.figure(figsize=(14, 8))

ax1 = fig.add_subplot(231)
ax1.imshow(
    np.sum(np.abs(insight.Ew), axis=-1),
    cmap="cubehelix",
    origin="lower",
    extent=(insight.x[0], insight.x[-1], insight.y[0], insight.y[-1]),
)
ax1.set_title("spectrally integrated amplitude, far field")
ax1.set_xlabel("x [mm]")
ax1.set_ylabel("y [mm]")

ax2 = fig.add_subplot(232)
ax2.imshow(
    np.sum(np.abs(insight.Ew_NF), axis=-1),
    cmap="cubehelix",
    origin="lower",
    extent=(insight.x_NF[0], insight.x_NF[-1], insight.y_NF[0], insight.y_NF[-1]),
)
ax2.set_title("spectrally integrated amplitude, near field")
ax2.set_xlabel("x [mm]")
ax2.set_ylabel("y [mm]")

ax3 = fig.add_subplot(233)
ax3.plot(
    insight.w,
    np.angle(
        insight.Ew_NF[np.abs(insight.y_NF - insight.yc_NF).argmin(), np.abs(insight.x_NF - insight.xc_NF).argmin(), :]
    ),
)
ax3.set_title("phase in near field beam center")
ax3.set_xlabel(r"$\omega$ [rad/fs]")
ax3.set_ylabel("rad")

ax4 = fig.add_subplot(234)
ax4.imshow(
    np.sum(np.abs(insight.Ew), axis=0),
    cmap="cubehelix",
    origin="lower",
    aspect="auto",
    extent=(insight.w[0], insight.w[-1], insight.x[0], insight.x[-1]),
)
ax4.set_title(r"SD$_x$ in focus")
ax4.set_ylabel("x [mm]")
ax4.set_xlabel(r"$\omega$ [rad/fs]")

ax5 = fig.add_subplot(235)
ax5.imshow(
    np.sum(np.abs(insight.Ew), axis=1),
    cmap="cubehelix",
    origin="lower",
    aspect="auto",
    extent=(insight.w[0], insight.w[-1], insight.x[0], insight.x[-1]),
)
ax5.set_title(r"SD$_y$ in focus")
ax5.set_xlabel(r"$\omega$ [rad/fs]")
ax5.set_ylabel("y [mm]")

ax6 = fig.add_subplot(236)
# sum just over the main beam spot to extract the spectrum
ax6.plot(
    insight.w,
    np.sum(
        np.sum(
            np.abs(
                insight.Ew[
                    np.abs(insight.y - insight.yc + 2 * insight.waist).argmin() : np.abs(
                        insight.y - insight.yc - 2 * insight.waist
                    ).argmin(),
                    np.abs(insight.x - insight.xc + 2 * insight.waist).argmin() : np.abs(
                        insight.x - insight.xc - 2 * insight.waist
                    ).argmin(),
                    :,
                ]
            )
            ** 2,
            axis=0,
        ),
        axis=0,
    ),
)
ax6.set_title("spectral intensity")
ax6.set_xlabel(r"$\omega$ [rad/fs]")
ax6.set_ylabel("arb. units")

plt.subplots_adjust(hspace=0.3, wspace=0.3)
plt.show()

## Correct the data
### Adjust beam compression and add dispersion parameters
Before going on with any calculations, the phase has to be corrected. Insight reconstructs the amplitude of the far field beam aswell as the phase, up to an unknown global phase for every frequency.
For an estimation of this global phase, perfect compression is assumed in the (near field) beam center. Thus, the phase is extracted in the beam center in dependence of the frequency and substracted globally (i.e. from the measured phase in dependence of the frequency at every space point).
Here, one also has the possibility to add dispersion parameters such as group velocity dispersion (`GVD`) and third order dispersion (`TOD`) (both are set to 0 by default).

In [ ]:
insight.correct_phase()

In [ ]:
plt.plot(
    insight.w,
    np.angle(
        insight.Ew_NF[np.abs(insight.y_NF - insight.yc_NF).argmin(), np.abs(insight.x_NF - insight.xc_NF).argmin(), :]
    ),
)
plt.title("corrected phase in near field beam center")
plt.xlabel(r"$\omega$ [rad/fs]")
plt.ylabel("rad")
plt.show()

### Correct ugly spots in the near field (optional)
Sometimes, the amplitude in the near field looks weird, showing some (unphysical) peaks or holes. These can cause artifacts in the far field and will thus be smoothened out.

In [ ]:
ugly_x = 0.0  # mm
ugly_y = 0.0  # mm
insight.correct_ugly_spot_in_nf(ugly_x, ugly_y)

In [ ]:
fig = plt.figure(figsize=(10, 4))

ax1 = fig.add_subplot(121)
ax1.imshow(
    np.sum(np.abs(insight.Ew), axis=-1),
    cmap="cubehelix",
    origin="lower",
    extent=(insight.x[0], insight.x[-1], insight.y[0], insight.y[-1]),
)
ax1.set_title("spectrally integrated amplitude, far field")
ax1.set_xlabel("x [mm]")
ax1.set_ylabel("y [mm]")

ax2 = fig.add_subplot(122)
ax2.imshow(
    np.sum(np.abs(insight.Ew_NF), axis=-1),
    cmap="cubehelix",
    origin="lower",
    extent=(insight.x_NF[0], insight.x_NF[-1], insight.y_NF[0], insight.y_NF[-1]),
)
ax2.set_title("spectrally integrated amplitude, near field")
ax2.set_xlabel("x [mm]")
ax2.set_ylabel("y [mm]")

plt.show()

### Center the near field beam spot (optional)
When the near field beam spot is not centered (please check the center coordinates above), the far field will propagate obliquely instead of straight ahead. Centering the near field prevents this.

In [ ]:
insight.shift_nf_to_center()

In [ ]:
fig = plt.figure(figsize=(14, 4))

ax1 = fig.add_subplot(131)
ax1.imshow(
    np.sum(np.abs(insight.Ew), axis=-1),
    cmap="cubehelix",
    origin="lower",
    extent=(insight.x[0], insight.x[-1], insight.y[0], insight.y[-1]),
)
ax1.set_title("spectrally integrated amplitude, far field")
ax1.set_xlabel("x [mm]")
ax1.set_ylabel("y [mm]")

ax2 = fig.add_subplot(132)
ax2.imshow(
    np.sum(np.abs(insight.Ew_NF), axis=-1),
    cmap="cubehelix",
    origin="lower",
    extent=(insight.x_NF[0], insight.x_NF[-1], insight.y_NF[0], insight.y_NF[-1]),
)
ax2.set_title("spectrally integrated amplitude, near field")
ax2.set_xlabel("x [mm]")
ax2.set_ylabel("y [mm]")
plt.show()

## Measure dispersion parameters
You can measure angular dispersion and spatial dispersion in far- and near field.
For a consistency check of those values, you can check those relations: 
\begin{equation}
\begin{aligned}
AD_{FF} &= - SD_{NF} / f - AD_{NF} \\
SD_{FF} &= f \cdot AD_{NF}
\end{aligned}
\end{equation}
$f$ is the focal length.

In [ ]:
insight.measure_ad_in_nf()
insight.measure_ad_in_ff()
insight.measure_sd_in_nf()
insight.measure_sd_in_ff()

## Add an aperture in the mid field (optional)
Here, you can apply an aperture to the beam, located in the mid field. The algorithm which propagates the beam to the mid field uses paraxial approximation; so please make sure to put the aperture not too close to the focal plane. 

In [ ]:
d = 1000  # mm, distance from focal plane to aperture
R = 38.5 / 2  # mm, aperture radius
insight.aperture_in_mf(d, R)

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.imshow(
    np.sum(np.abs(insight.Ew), axis=-1),
    cmap="cubehelix",
    origin="lower",
    extent=(insight.x[0], insight.x[-1], insight.y[0], insight.y[-1]),
)
ax1.set_title("spectrally integrated amplitude, far field")
ax1.set_xlabel("x [mm]")
ax1.set_ylabel("y [mm]")
plt.show()

## Propagate
Now the far field data is ready to be propagated. For that, the angular spectrum method is used. 
Watch out not to propagate too far, since then the growing beam diameter could reach the transversal window borders and thus cause fourier transform artifacts.

In [ ]:
z = -2  # mm
Ew_prop = insight.propagate(z)

In [ ]:
plt.imshow(
    np.sum(np.abs(Ew_prop), axis=-1),
    cmap="cubehelix",
    origin="lower",
    extent=(insight.x[0], insight.x[-1], insight.y[0], insight.y[-1]),
)
plt.title("spectrally integrated amplitude, propagated to %.2f mm" % (z))
plt.xlabel("x [mm]")
plt.ylabel("y [mm]")
plt.show()

## Transform to the time domain
The far field data will be transformed to the time domain via a 1D fourier transformation. This takes a while, since the spectrum has to be extended and the field data extrapolated. One can adjust the number of samples per wavelength, which is set to 10 by default. \
**Attention:** it is recommeded to adjust the time sampling to the PIConGPU simulation timestep! \
The real part of the resulting complex matrix `insight.Et` will be the field needed for PIConGPU and its absolute value is the envelope, which can be used for further analysis.

In [ ]:
insight.to_time_domain(Ew_prop)

In [ ]:
fig = plt.figure(figsize=(6, 9))

ax1 = fig.add_subplot(311)
ax1.imshow(
    np.sum(np.abs(insight.Et), axis=0),
    cmap="cubehelix",
    origin="lower",
    aspect="auto",
    extent=(insight.t[0], insight.t[-1], insight.x[0], insight.x[-1]),
)
ax1.set_ylabel("x [mm]")
ax1.set_title("transversally integrated field")

ax2 = fig.add_subplot(312)
ax2.imshow(
    np.sum(np.abs(insight.Et), axis=1),
    cmap="cubehelix",
    origin="lower",
    aspect="auto",
    extent=(insight.t[0], insight.t[-1], insight.x[0], insight.x[-1]),
)
ax2.set_ylabel("y [mm]")

ax3 = fig.add_subplot(313)
Et_center = insight.Et[np.abs(insight.y - insight.yc).argmin(), np.abs(insight.x - insight.xc).argmin(), :]
ax3.plot(insight.t, np.real(Et_center), label="real part")
ax3.plot(insight.t, np.abs(Et_center), label="envelope")
ax3.set_xlabel("t [fs]")
ax3.set_ylabel("field strength [arb. units]")
ax3.set_title("field in beam center")
plt.legend()

plt.subplots_adjust(hspace=0.25)
plt.show()

## Save data to openPMD
The data is now nearly ready te be used as FromOpenPMDPulse input. The amplitude of the pulse in the time domain still has to be corrected (= scaled to the actual beam energy in Joule) before saving it to an openPMD file at the provided destination path.
Please pay attention to the size of the field data chunk: its real part will be stored on each used GPU as a whole, but their memory is limited. To reduce the chunk size, one can trim the edges by `crop_x`, `crop_y` (in mm) and `crop_t` (in fs).

In [ ]:
E = 4.5  # J, beam energy
pol = "x"  # polarisation axis (can be 'x' or 'y')
crop_x = 0.3  # mm, trim transversal x axis
crop_y = 0.3  # mm, trim transversal y axis
crop_t = 100  # fs, trim time axis

insight.save_to_openPMD("", "insightData_prepared%T.h5", E, pol, crop_x, crop_y, crop_t)